In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE
from rbot.chart import Chart

In [6]:
class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            self.K = param_K                           # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*60*2
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        ohlcv_df = session.ohlcv(60*60*2, 6)       # ６本の足を取得。 最新は６番目。

        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean()      #　過去５本足のレンジの平均値

        # Long/Short判定
        detect_short = range_width * self.K < ohlcv_latest["high"][0] - ohlcv_latest["open"][0]
        detect_long  = range_width * self.K < ohlcv_latest["open"][0] - ohlcv_latest["low"][0]

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_position_size:
            #    print("makeorder")                                    
                if not session.short_position_size:

                    session.place_order("Buy", session.best_buy_price, 0.01, 600, "Open Long")    
                else:

                    session.place_order("Buy", session.best_buy_price, 0.02, 600, "doten Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.short_position_size:
                #print("makeorder")                                                      
                if not session.long_position_size:
                    session.place_order("Sell", session.best_sell_price, 0.01, 600, "Open Short") 
                else:
                    session.place_order("Sell", session.best_sell_price, 0.02, 600, "Doten Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [7]:
back_tester = BackTester("BN", "BTCBUSD", False)
back_tester.maker_fee_rate = 0.1 * 0.01  # 0.1%
back_tester.maker_fee_rate

0.001

In [8]:


print("start")
r = back_tester.run(Agent(), rbot.DAYS_BEFORE(10), 0)
#r = back_tester.run(Agent(), 0, 0)
print("end")




start
end


In [9]:
import rbot
df = rbot.result_to_df(r)

In [10]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-01 14:00:00+00:00,2022-12-01 14:10:00.127000+00:00,0000-0001,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,17103.24,0.01,171.0324,0.0000,0.000000,0.000000,0.00,Open Short,0.000000,0.00
2022-12-02 14:00:00+00:00,2022-12-02 14:00:00.070000+00:00,0000-0002,0,Buy,True,Open,16997.89,0.01,169.9789,0.00,...,16997.89,0.01,169.9789,0.0000,0.169979,-0.169979,0.01,Open Long,-0.169979,0.01
2022-12-04 02:00:00+00:00,2022-12-04 02:00:00.601000+00:00,0000-0003,0,Sell,True,Close,16997.89,0.01,169.9789,16888.07,...,16888.07,0.01,168.8807,-1.0982,0.168881,-1.267081,-0.01,Doten Short,-1.437060,0.00
2022-12-04 02:00:00+00:00,2022-12-04 02:00:00.601000+00:00,0000-0003,1,Sell,True,Open,16888.07,0.01,168.8807,0.00,...,16888.07,0.01,168.8807,0.0000,0.168881,-0.168881,-0.01,Doten Short,-1.605940,-0.01
2022-12-05 16:00:00+00:00,2022-12-05 16:00:00.034000+00:00,0000-0004,0,Buy,True,Close,16888.07,0.01,168.8807,17234.33,...,17234.33,0.01,172.3433,-3.4626,0.172343,-3.634943,0.01,doten Long,-5.240884,0.00
2022-12-05 16:00:00+00:00,2022-12-05 16:00:00.034000+00:00,0000-0004,1,Buy,True,Open,17234.33,0.01,172.3433,0.00,...,17234.33,0.01,172.3433,0.0000,0.172343,-0.172343,0.01,doten Long,-5.413227,0.01


In [11]:
market = Market.open("BN", "BTCBUSD")
ohlcv = market.ohlcv(rbot.NOW()-rbot.DAYS(12), 0, 60*60)


In [12]:
chart = Chart(800, 400, ohlcv)
chart.show()

Loading BokehJS ...

main


In [10]:
df = result_to_df(r)
df

NameError: name 'result_to_df' is not defined

In [ ]:
bn = BinanceMarket("BTCBUSD")
#bn.download(15)


In [ ]:
bn

In [ ]:
rbot.init_debug_log()
bn.download(10)

In [ ]:
bn.info()

In [ ]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

In [ ]:
bn.ohlcv(0, 0, 600)

In [ ]:
result_to_df(r)

In [ ]:
r[0].update_time